In [1]:
import itertools
import numpy as np
import pandas as pd

sentences = '''
sam is red
hannah not red
hannah is green
bob is green
bob not red
sam not green
sarah is red
sarah not green'''.strip().split('\n')
is_green = np.asarray([[0, 1, 1, 1, 1, 0, 0, 0]], dtype='int32').T

lemma = lambda x: x.strip().lower().split(' ')
sentences_lemmatized = [lemma(sentence) for sentence in sentences]
words = set(itertools.chain(*sentences_lemmatized))

In [2]:
for it in itertools.chain(sentences_lemmatized):
    print(it)

['sam', 'is', 'red']
['hannah', 'not', 'red']
['hannah', 'is', 'green']
['bob', 'is', 'green']
['bob', 'not', 'red']
['sam', 'not', 'green']
['sarah', 'is', 'red']
['sarah', 'not', 'green']


In [3]:
words

{'bob', 'green', 'hannah', 'is', 'not', 'red', 'sam', 'sarah'}

In [4]:
for w in words:
    print(w)

not
hannah
red
sam
is
green
bob
sarah


In [5]:
for i, w in enumerate(words):
    print(w)

not
hannah
red
sam
is
green
bob
sarah


In [6]:
list(words)

['not', 'hannah', 'red', 'sam', 'is', 'green', 'bob', 'sarah']

In [7]:
sentences_lemmatized

[['sam', 'is', 'red'],
 ['hannah', 'not', 'red'],
 ['hannah', 'is', 'green'],
 ['bob', 'is', 'green'],
 ['bob', 'not', 'red'],
 ['sam', 'not', 'green'],
 ['sarah', 'is', 'red'],
 ['sarah', 'not', 'green']]

In [8]:
word2idx = dict((v, i) for i, v in enumerate(words))
idx2word = list(words)

In [9]:
word2idx

{'bob': 6,
 'green': 5,
 'hannah': 1,
 'is': 4,
 'not': 0,
 'red': 2,
 'sam': 3,
 'sarah': 7}

In [10]:
idx2word

['not', 'hannah', 'red', 'sam', 'is', 'green', 'bob', 'sarah']

In [11]:
to_idx = lambda x: [word2idx[word] for word in x]
sentences_idx = [to_idx(sentence) for sentence in sentences_lemmatized]
sentences_array = np.asarray(sentences_idx, dtype='int32')

In [12]:
sentences_idx[2]

[1, 4, 5]

In [13]:
sentences_array[2]

array([1, 4, 5], dtype=int32)

In [15]:
sentence_maxlen = 3
n_words = len(words)
n_embed_dims = 3

# put together a model to predict
from keras.layers import Input, Embedding, merge, Flatten, SimpleRNN
from keras.models import Model

input_sentence = Input(shape=(sentence_maxlen,), dtype='int32')
input_embedding = Embedding(n_words, n_embed_dims)(input_sentence)
color_prediction = SimpleRNN(1)(input_embedding)

In [17]:
predict_green = Model(input=[input_sentence], output=[color_prediction])
predict_green.compile(optimizer='sgd', loss='binary_crossentropy')

# fit the model to predict what color each person is
predict_green.fit([sentences_array], [is_green], nb_epoch=5000, verbose=1)

/Users/yan/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  """Entry point for launching an IPython kernel.
/Users/yan/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Epoch 1/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 2/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 3/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 4/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 5/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 6/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 7/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 8/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 9/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 10/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 11/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 12/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 13/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 14/5000
8/8 [==============================] - 0s - loss: 0.0016
Epoch 15/5000
8

8/8 [==============================] - 0s - loss: 0.0015
Epoch 117/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 118/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 119/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 120/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 121/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 122/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 123/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 124/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 125/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 126/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 127/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 128/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 129/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 1

Epoch 230/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 231/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 232/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 233/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 234/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 235/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 236/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 237/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 238/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 239/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 240/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 241/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 242/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 243/5000
8/8 [==============================] - 0s - loss:

8/8 [==============================] - 0s - loss: 0.0015
Epoch 344/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 345/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 346/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 347/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 348/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 349/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 350/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 351/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 352/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 353/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 354/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 355/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 356/5000
8/8 [==============================] - 0s - loss: 0.0015
Epoch 3

...

8/8 [==============================] - 0s - loss: 8.1765e-04
Epoch 4515/5000
8/8 [==============================] - 0s - loss: 8.1755e-04
Epoch 4516/5000
8/8 [==============================] - 0s - loss: 8.1745e-04
Epoch 4517/5000
8/8 [==============================] - 0s - loss: 8.1731e-04
Epoch 4518/5000
8/8 [==============================] - 0s - loss: 8.1724e-04
Epoch 4519/5000
8/8 [==============================] - 0s - loss: 8.1711e-04
Epoch 4520/5000
8/8 [==============================] - 0s - loss: 8.1703e-04
Epoch 4521/5000
8/8 [==============================] - 0s - loss: 8.1691e-04
Epoch 4522/5000
8/8 [==============================] - 0s - loss: 8.1680e-04
Epoch 4523/5000
8/8 [==============================] - 0s - loss: 8.1671e-04
Epoch 4524/5000
8/8 [==============================] - 0s - loss: 8.1655e-04
Epoch 4525/5000
8/8 [==============================] - 0s - loss: 8.1648e-04
Epoch 4526/5000
8/8 [==============================] - 0s - loss: 8.1638e-04
Epoch 4527/5000

8/8 [==============================] - 0s - loss: 8.0645e-04
Epoch 4621/5000
8/8 [==============================] - 0s - loss: 8.0635e-04
Epoch 4622/5000
8/8 [==============================] - 0s - loss: 8.0620e-04
Epoch 4623/5000
8/8 [==============================] - 0s - loss: 8.0613e-04
Epoch 4624/5000
8/8 [==============================] - 0s - loss: 8.0603e-04
Epoch 4625/5000
8/8 [==============================] - 0s - loss: 8.0591e-04
Epoch 4626/5000
8/8 [==============================] - 0s - loss: 8.0582e-04
Epoch 4627/5000
8/8 [==============================] - 0s - loss: 8.0572e-04
Epoch 4628/5000
8/8 [==============================] - 0s - loss: 8.0563e-04
Epoch 4629/5000
8/8 [==============================] - 0s - loss: 8.0550e-04
Epoch 4630/5000
8/8 [==============================] - 0s - loss: 8.0540e-04
Epoch 4631/5000
8/8 [==============================] - 0s - loss: 8.0531e-04
Epoch 4632/5000
8/8 [==============================] - 0s - loss: 8.0520e-04
Epoch 4633/5000

8/8 [==============================] - 0s - loss: 7.9554e-04
Epoch 4727/5000
8/8 [==============================] - 0s - loss: 7.9542e-04
Epoch 4728/5000
8/8 [==============================] - 0s - loss: 7.9530e-04
Epoch 4729/5000
8/8 [==============================] - 0s - loss: 7.9522e-04
Epoch 4730/5000
8/8 [==============================] - 0s - loss: 7.9513e-04
Epoch 4731/5000
8/8 [==============================] - 0s - loss: 7.9503e-04
Epoch 4732/5000
8/8 [==============================] - 0s - loss: 7.9490e-04
Epoch 4733/5000
8/8 [==============================] - 0s - loss: 7.9483e-04
Epoch 4734/5000
8/8 [==============================] - 0s - loss: 7.9470e-04
Epoch 4735/5000
8/8 [==============================] - 0s - loss: 7.9461e-04
Epoch 4736/5000
8/8 [==============================] - 0s - loss: 7.9452e-04
Epoch 4737/5000
8/8 [==============================] - 0s - loss: 7.9444e-04
Epoch 4738/5000
8/8 [==============================] - 0s - loss: 7.9429e-04
Epoch 4739/5000

8/8 [==============================] - 0s - loss: 7.8492e-04
Epoch 4833/5000
8/8 [==============================] - 0s - loss: 7.8483e-04
Epoch 4834/5000
8/8 [==============================] - 0s - loss: 7.8473e-04
Epoch 4835/5000
8/8 [==============================] - 0s - loss: 7.8464e-04
Epoch 4836/5000
8/8 [==============================] - 0s - loss: 7.8454e-04
Epoch 4837/5000
8/8 [==============================] - 0s - loss: 7.8442e-04
Epoch 4838/5000
8/8 [==============================] - 0s - loss: 7.8432e-04
Epoch 4839/5000
8/8 [==============================] - 0s - loss: 7.8423e-04
Epoch 4840/5000
8/8 [==============================] - 0s - loss: 7.8412e-04
Epoch 4841/5000
8/8 [==============================] - 0s - loss: 7.8405e-04
Epoch 4842/5000
8/8 [==============================] - 0s - loss: 7.8391e-04
Epoch 4843/5000
8/8 [==============================] - 0s - loss: 7.8385e-04
Epoch 4844/5000
8/8 [==============================] - 0s - loss: 7.8374e-04
Epoch 4845/5000

8/8 [==============================] - 0s - loss: 7.7456e-04
Epoch 4939/5000
8/8 [==============================] - 0s - loss: 7.7447e-04
Epoch 4940/5000
8/8 [==============================] - 0s - loss: 7.7436e-04
Epoch 4941/5000
8/8 [==============================] - 0s - loss: 7.7427e-04
Epoch 4942/5000
8/8 [==============================] - 0s - loss: 7.7419e-04
Epoch 4943/5000
8/8 [==============================] - 0s - loss: 7.7412e-04
Epoch 4944/5000
8/8 [==============================] - 0s - loss: 7.7403e-04
Epoch 4945/5000
8/8 [==============================] - 0s - loss: 7.7391e-04
Epoch 4946/5000
8/8 [==============================] - 0s - loss: 7.7383e-04
Epoch 4947/5000
8/8 [==============================] - 0s - loss: 7.7370e-04
Epoch 4948/5000
8/8 [==============================] - 0s - loss: 7.7362e-04
Epoch 4949/5000
8/8 [==============================] - 0s - loss: 7.7350e-04
Epoch 4950/5000
8/8 [==============================] - 0s - loss: 7.7344e-04
Epoch 4951/5000

In [33]:
embeddings = predict_green.layers[1].get_weights()
embeddings[0]

array([[ 0.02849239, -0.02936248,  0.02896237],
       [-0.09560233,  0.27610442,  0.37975305],
       [-0.05268165,  0.27214497,  0.42261177],
       [ 0.1208114 , -0.40689909, -0.41520679],
       [-0.0314412 ,  0.00850408, -0.01338043],
       [-0.08929905,  0.31607234,  0.37800881],
       [-0.04697246,  0.26065624,  0.38214168],
       [ 0.10527284, -0.39019316, -0.43702513]], dtype=float32)

In [32]:
embeddings = predict_green.layers[1].get_weights()

# print out the embedding vector associated with each word
for i in range(n_words):
	print('{}: {}'.format(idx2word[i], embeddings[0][i]))

not: [ 0.02849239 -0.02936248  0.02896237]
hannah: [-0.09560233  0.27610442  0.37975305]
red: [-0.05268165  0.27214497  0.42261177]
sam: [ 0.1208114  -0.40689909 -0.41520679]
is: [-0.0314412   0.00850408 -0.01338043]
green: [-0.08929905  0.31607234  0.37800881]
bob: [-0.04697246  0.26065624  0.38214168]
sarah: [ 0.10527284 -0.39019316 -0.43702513]


In [35]:
rng = np.random.RandomState(42)

# parameters
input_dims, output_dims = 10, 1
sequence_length = 20
n_test = 10

# generate some random data to train on
get_rand = lambda *shape: np.asarray(rng.rand(*shape) > 0.5, dtype='float32')
X_data = np.asarray([get_rand(sequence_length, input_dims) for _ in range(n_test)])
y_data = np.asarray([get_rand(output_dims,) for _ in range(n_test)])

# put together rnn models
from keras.layers import Input
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Model
#import theano

input_sequence = Input(shape=(sequence_length, input_dims,), dtype='float32')

vanilla = SimpleRNN(output_dims, return_sequences=False)(input_sequence)
lstm = LSTM(output_dims, return_sequences=False)(input_sequence)
gru = GRU(output_dims, return_sequences=False)(input_sequence)
rnns = [vanilla, lstm, gru]

# train the models
for rnn in rnns:
    model = Model(input=[input_sequence], output=rnn)
    model.compile(optimizer='rmsprop', loss='binary_crossentropy')
    model.fit([X_data], [y_data], nb_epoch=1000)

/Users/yan/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("si..., inputs=[<tf.Tenso...)`
/Users/yan/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:30: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/1000
10/10 [==============================] - 0s - loss: 1.2645
Epoch 2/1000
10/10 [==============================] - 0s - loss: 1.2503
Epoch 3/1000
10/10 [==============================] - 0s - loss: 1.2401
Epoch 4/1000
10/10 [==============================] - 0s - loss: 1.2318
Epoch 5/1000
10/10 [==============================] - 0s - loss: 1.2244
Epoch 6/1000
10/10 [==============================] - 0s - loss: 1.2177
Epoch 7/1000
10/10 [==============================] - 0s - loss: 1.2116
Epoch 8/1000
10/10 [==============================] - 0s - loss: 1.2058
Epoch 9/1000
10/10 [==============================] - 0s - loss: 1.2002
Epoch 10/1000
10/10 [==============================] - 0s - loss: 1.1950
Epoch 11/1000
10/10 [==============================] - 0s - loss: 1.1899
Epoch 12/1000
10/10 [==============================] - 0s - loss: 1.1850
Epoch 13/1000
10/10 [==============================] - 0s - loss: 1.1802
Epoch 14/1000
10/10 [==============================] - 0s - 

10/10 [==============================] - 0s - loss: 0.7968
Epoch 114/1000
10/10 [==============================] - 0s - loss: 0.7933
Epoch 115/1000
10/10 [==============================] - 0s - loss: 0.7898
Epoch 116/1000
10/10 [==============================] - 0s - loss: 0.7862
Epoch 117/1000
10/10 [==============================] - 0s - loss: 0.7827
Epoch 118/1000
10/10 [==============================] - 0s - loss: 0.7792
Epoch 119/1000
10/10 [==============================] - 0s - loss: 0.7758
Epoch 120/1000
10/10 [==============================] - 0s - loss: 0.7723
Epoch 121/1000
10/10 [==============================] - 0s - loss: 0.7688
Epoch 122/1000
10/10 [==============================] - 0s - loss: 0.7653
Epoch 123/1000
10/10 [==============================] - 0s - loss: 0.7619
Epoch 124/1000
10/10 [==============================] - 0s - loss: 0.7584
Epoch 125/1000
10/10 [==============================] - 0s - loss: 0.7550
Epoch 126/1000
10/10 [==============================]

10/10 [==============================] - 0s - loss: 0.4549
Epoch 224/1000
10/10 [==============================] - 0s - loss: 0.4521
Epoch 225/1000
10/10 [==============================] - 0s - loss: 0.4494
Epoch 226/1000
10/10 [==============================] - 0s - loss: 0.4467
Epoch 227/1000
10/10 [==============================] - 0s - loss: 0.4440
Epoch 228/1000
10/10 [==============================] - 0s - loss: 0.4414
Epoch 229/1000
10/10 [==============================] - 0s - loss: 0.4387
Epoch 230/1000
10/10 [==============================] - 0s - loss: 0.4360
Epoch 231/1000
10/10 [==============================] - 0s - loss: 0.4333
Epoch 232/1000
10/10 [==============================] - 0s - loss: 0.4307
Epoch 233/1000
10/10 [==============================] - 0s - loss: 0.4280
Epoch 234/1000
10/10 [==============================] - 0s - loss: 0.4254
Epoch 235/1000
10/10 [==============================] - 0s - loss: 0.4227
Epoch 236/1000
10/10 [==============================]

10/10 [==============================] - 0s - loss: 0.2141
Epoch 334/1000
10/10 [==============================] - 0s - loss: 0.2123
Epoch 335/1000
10/10 [==============================] - 0s - loss: 0.2105
Epoch 336/1000
10/10 [==============================] - 0s - loss: 0.2088
Epoch 337/1000
10/10 [==============================] - 0s - loss: 0.2070
Epoch 338/1000
10/10 [==============================] - 0s - loss: 0.2053
Epoch 339/1000
10/10 [==============================] - 0s - loss: 0.2035
Epoch 340/1000
10/10 [==============================] - 0s - loss: 0.2018
Epoch 341/1000
10/10 [==============================] - 0s - loss: 0.2001
Epoch 342/1000
10/10 [==============================] - 0s - loss: 0.1983
Epoch 343/1000
10/10 [==============================] - 0s - loss: 0.1966
Epoch 344/1000
10/10 [==============================] - 0s - loss: 0.1949
Epoch 345/1000
10/10 [==============================] - 0s - loss: 0.1931
Epoch 346/1000
10/10 [==============================]

...

10/10 [==============================] - 0s - loss: 4.8354
Epoch 554/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 555/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 556/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 557/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 558/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 559/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 560/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 561/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 562/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 563/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 564/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 565/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 566/1000
10/10 [==============================]

10/10 [==============================] - 0s - loss: 4.8354
Epoch 664/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 665/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 666/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 667/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 668/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 669/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 670/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 671/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 672/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 673/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 674/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 675/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 676/1000
10/10 [==============================]

10/10 [==============================] - 0s - loss: 4.8354
Epoch 774/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 775/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 776/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 777/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 778/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 779/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 780/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 781/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 782/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 783/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 784/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 785/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 786/1000
10/10 [==============================]

10/10 [==============================] - 0s - loss: 4.8354
Epoch 884/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 885/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 886/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 887/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 888/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 889/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 890/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 891/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 892/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 893/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 894/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 895/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 896/1000
10/10 [==============================]

10/10 [==============================] - 0s - loss: 4.8354
Epoch 994/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 995/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 996/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 997/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 998/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 999/1000
10/10 [==============================] - 0s - loss: 4.8354
Epoch 1000/1000
10/10 [==============================] - 0s - loss: 4.8354


## 20 Newgroups Classification

In [44]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model


BASE_DIR = '.'
GLOVE_DIR = BASE_DIR + '/assignment4/download/dwr/'
TEXT_DATA_DIR = BASE_DIR + '/20_newsgroup/'
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2


In [43]:
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                if sys.version_info < (3,):
                    f = open(fpath)
                else:
                    f = open(fpath, encoding='latin-1')
                t = f.read()
                i = t.find('\n\n')  # skip header
                if 0 < i:
                    t = t[i:]
                texts.append(t)
                f.close()
                labels.append(label_id)

print('Found %s texts.' % len(texts))

Found 19997 texts.


### Then we can format our text samples and labels into tensors that can be fed into a neural network. To do this, we will rely on Keras utilities keras.preprocessing.text.Tokenizer and keras.preprocessing.sequence.pad_sequences.

In [45]:
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

/Users/yan/anaconda/envs/py35/lib/python3.5/site-packages/keras/preprocessing/text.py:145: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 174074 unique tokens.


In [46]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)


Shape of data tensor: (19997, 1000)
Shape of label tensor: (19997, 20)


In [47]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

### Next, we compute an index mapping words to known embeddings, by parsing the data dump of pre-trained embeddings:

In [48]:
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


### At this point we can leverage our embedding_index dictionary and our word_index to compute our embedding matrix:

In [49]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

### We load this embedding matrix into an Embedding layer. Note that we set trainable=False to prevent the weights from being updated during training.

In [50]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

#### We can also test how well we would have performed by not using pre-trained word embeddings, but instead initializing our Embedding layer from scratch and learning its weights during training. We just need to replace our Embedding layer with the following:



In [52]:
#embedding_layer = Embedding(len(word_index) + 1,
#                            EMBEDDING_DIM,
#                            input_length=MAX_SEQUENCE_LENGTH)

### An Embedding layer should be fed sequences of integers, i.e. a 2D input of shape (samples, indices). These input sequences should be padded so that they all have the same length in a batch of input data (although an Embedding layer is capable of processing sequence of heterogenous length, if you don't pass an explicit input_length argument to the layer).

All that the Embedding layer does is to map the integer inputs to the vectors found at the corresponding index in the embedding matrix, i.e. the sequence [1, 2] would be converted to [embeddings[1], embeddings[2]]. This means that the output of the Embedding layer will be a 3D tensor of shape (samples, sequence_length, embedding_dim).

Training a 1D convnet

Finally we can then build a small 1D convnet to solve our classification problem:

In [51]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=2, batch_size=128)

Train on 15998 samples, validate on 3999 samples
Epoch 1/2
15998/15998 [==============================] - 158s - loss: 2.4113 - acc: 0.2060 - val_loss: 1.7617 - val_acc: 0.3873
Epoch 2/2
15998/15998 [==============================] - 207s - loss: 1.5605 - acc: 0.4549 - val_loss: 1.3770 - val_acc: 0.5149


# Keras LSTM for bAbI question answering

In [55]:
'''Trains a memory network on the bAbI dataset.

References:
- Jason Weston, Antoine Bordes, Sumit Chopra, Tomas Mikolov, Alexander M. Rush,
  "Towards AI-Complete Question Answering: A Set of Prerequisite Toy Tasks",
  http://arxiv.org/abs/1502.05698

- Sainbayar Sukhbaatar, Arthur Szlam, Jason Weston, Rob Fergus,
  "End-To-End Memory Networks",
  http://arxiv.org/abs/1503.08895

Reaches 98.6% accuracy on task 'single_supporting_fact_10k' after 120 epochs.
Time per epoch: 3s on CPU (core i7).
'''
from __future__ import print_function

from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, \
dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from functools import reduce
import tarfile
import numpy as np
import re

In [56]:
def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.

    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)?', sent) if x.strip()]


def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format

    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.

    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not \
            max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[answer]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))


In [57]:
try:
    path = get_file('babi-tasks-v1-2.tar.gz', \
                    origin='https://s3.amazonaws.com/text-datasets/babi_tasks_1-20_v1-2.tar.gz')
except:
    print('Error downloading dataset, please download it manually:\n'
          '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
          '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

11558912/11745123 [============================>.] - ETA: 0s

In [58]:
%who

Activation	 BASE_DIR	 Conv1D	 Dense	 Dropout	 EMBEDDING_DIM	 Embedding	 Flatten	 GLOVE_DIR	 
GRU	 Input	 LSTM	 MAX_NB_WORDS	 MAX_SEQUENCE_LENGTH	 MaxPooling1D	 Model	 Permute	 Sequential	 
SimpleRNN	 TEXT_DATA_DIR	 Tokenizer	 VALIDATION_SPLIT	 X_data	 add	 coefs	 color_prediction	 concatenate	 
data	 dot	 embedded_sequences	 embedding_layer	 embedding_matrix	 embedding_vector	 embeddings	 embeddings_index	 f	 
fname	 fpath	 get_file	 get_rand	 get_stories	 gru	 i	 idx2word	 indices	 
input_dims	 input_embedding	 input_sentence	 input_sequence	 is_green	 it	 itertools	 label_id	 labels	 
labels_index	 lemma	 line	 lstm	 merge	 model	 n_embed_dims	 n_test	 n_words	 
name	 nb_validation_samples	 np	 os	 output_dims	 pad_sequences	 parse_stories	 path	 pd	 
predict_green	 preds	 print_function	 re	 reduce	 rng	 rnn	 rnns	 sentence_maxlen	 
sentences	 sentences_array	 sentences_idx	 sentences_lemmatized	 sequence_input	 sequence_length	 sequences	 sys	 t	 
tar	 tarfile	 texts	 to_categorica

In [59]:
challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

Extracting stories for the challenge: single_supporting_fact_10k


/Users/yan/anaconda/envs/py35/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [60]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

In [61]:
print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')
print('Vectorizing the word sequences...')

-
Vocab size: 22 unique words
Story max length: 68 words
Query max length: 4 words
Number of training stories: 10000
Number of test stories: 1000
-
Here's what a "story" tuple looks like (input, query, answer):
(['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'John', 'went', 'to', 'the', 'hallway', '.'], ['Where', 'is', 'Mary', '?'], 'bathroom')
-
Vectorizing the word sequences...


In [62]:
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories,
                                                               word_idx,
                                                               story_maxlen,
                                                               query_maxlen)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories,
                                                            word_idx,
                                                            story_maxlen,
                                                            query_maxlen)

In [63]:
print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')
print('Compiling...')


-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10000, 68)
inputs_test shape: (1000, 68)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10000, 4)
queries_test shape: (1000, 4)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10000, 22)
answers_test shape: (1000, 22)
-
Compiling...


In [64]:
# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

In [65]:
# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [66]:
# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

In [67]:
# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

In [ ]:
# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=120,
          validation_data=([inputs_test, queries_test], answers_test))

Train on 10000 samples, validate on 1000 samples
Epoch 1/120
10000/10000 [==============================] - 8s - loss: 1.9513 - acc: 0.1706 - val_loss: 1.8007 - val_acc: 0.1490
Epoch 2/120
10000/10000 [==============================] - 7s - loss: 1.7307 - acc: 0.2580 - val_loss: 1.6016 - val_acc: 0.3590
Epoch 3/120
10000/10000 [==============================] - 7s - loss: 1.5917 - acc: 0.3765 - val_loss: 1.5301 - val_acc: 0.3980
Epoch 4/120
10000/10000 [==============================] - 7s - loss: 1.5323 - acc: 0.4038 - val_loss: 1.4921 - val_acc: 0.4360
Epoch 5/120
10000/10000 [==============================] - 7s - loss: 1.5082 - acc: 0.4110 - val_loss: 1.4650 - val_acc: 0.4440
Epoch 6/120
10000/10000 [==============================] - 7s - loss: 1.4816 - acc: 0.4295 - val_loss: 1.4429 - val_acc: 0.4520
Epoch 7/120
10000/10000 [==============================] - 8s - loss: 1.4435 - acc: 0.4508 - val_loss: 1.3743 - val_acc: 0.4870
Epoch 8/120
10000/10000 [==============================